In [19]:
import os
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import time
import copy

from torch.optim import lr_scheduler
from torchvision import datasets
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter

In [20]:
batch_size = 16
epochs = 100
input_path = '/home/piai/바탕화면/hanja_crop_50/new100/new100_split/'
writer = SummaryWriter('/home/piai/바탕화면/hanja_crop_50/new100/new100_split/')

In [21]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(input_path, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'validation']}


In [22]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [23]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes = 4893)
model.to(device)

Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [24]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [25]:
best_model_weights = copy.deepcopy(model.state_dict())
best_acc = 0.0

In [26]:
for epoch in range(epochs):
    since = time.time()
    print('Epoch {}/{}'.format(epoch, epochs - 1))
    print('-' * 10)

    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        if phase == 'train':
            scheduler.step()

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        torch.save(model.state_dict(), '/home/piai/바탕화면/hanja_crop_50/new100/weights/effi_re_ver_weights_class_{}.pth'.format(epoch))
        if phase == 'validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_weights = copy.deepcopy(model.state_dict())
    time_elapsed = time.time() - since
    print("time : ", time_elapsed)
    print()

Epoch 0/99
----------
train Loss: 1.3282 Acc: 0.7579
validation Loss: 0.2878 Acc: 0.9307
time :  3905.134184360504

Epoch 1/99
----------
train Loss: 0.2679 Acc: 0.9337
validation Loss: 0.2190 Acc: 0.9453
time :  3880.615396976471

Epoch 2/99
----------
train Loss: 0.2034 Acc: 0.9487
validation Loss: 0.1978 Acc: 0.9512
time :  4157.596769809723

Epoch 3/99
----------
train Loss: 0.1719 Acc: 0.9554
validation Loss: 0.1839 Acc: 0.9538
time :  4029.8900043964386

Epoch 4/99
----------
train Loss: 0.1493 Acc: 0.9600
validation Loss: 0.1777 Acc: 0.9541
time :  3951.2796773910522

Epoch 5/99
----------
train Loss: 0.1324 Acc: 0.9640
validation Loss: 0.1671 Acc: 0.9567
time :  3816.9351868629456

Epoch 6/99
----------
train Loss: 0.1196 Acc: 0.9665
validation Loss: 0.1734 Acc: 0.9560
time :  4031.6974215507507

Epoch 7/99
----------
train Loss: 0.0546 Acc: 0.9816
validation Loss: 0.1265 Acc: 0.9652
time :  4002.354947090149

Epoch 8/99
----------
train Loss: 0.0416 Acc: 0.9853
validation Loss

train Loss: 0.0120 Acc: 0.9965
validation Loss: 0.1482 Acc: 0.9648
time :  3813.2382357120514

Epoch 71/99
----------
train Loss: 0.0119 Acc: 0.9966
validation Loss: 0.1482 Acc: 0.9648
time :  3840.7995710372925

Epoch 72/99
----------
train Loss: 0.0119 Acc: 0.9966
validation Loss: 0.1480 Acc: 0.9648
time :  3841.2134561538696

Epoch 73/99
----------
train Loss: 0.0120 Acc: 0.9964
validation Loss: 0.1482 Acc: 0.9647
time :  3843.619783639908

Epoch 74/99
----------
train Loss: 0.0119 Acc: 0.9966
validation Loss: 0.1478 Acc: 0.9647
time :  3846.762119293213

Epoch 75/99
----------
train Loss: 0.0118 Acc: 0.9966
validation Loss: 0.1480 Acc: 0.9648
time :  4111.8703780174255

Epoch 76/99
----------
train Loss: 0.0119 Acc: 0.9965
validation Loss: 0.1482 Acc: 0.9649
time :  4224.474766969681

Epoch 77/99
----------
train Loss: 0.0119 Acc: 0.9966
validation Loss: 0.1481 Acc: 0.9649
time :  4224.663674354553

Epoch 78/99
----------
train Loss: 0.0117 Acc: 0.9967
validation Loss: 0.1485 Acc: 

In [27]:
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

torch.save(best_model_weights, '/home/piai/바탕화면/hanja_crop_50/new100/weights/best_weights_effi_re_ver.pth')

Training complete in 63m 38s
Best val Acc: 0.965454
